In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


from pyspark.sql.functions import col, to_date, avg, max, min, current_timestamp

# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================
# PASTE YOUR ABFSS PATH HERE AGAIN 👇
LAKEHOUSE_ROOT_PATH = "abfss://DE_Project_Portfolio@onelake.dfs.fabric.microsoft.com/Lh_Medallion.Lakehouse/Files"

# Input (Silver) and Output (Gold) Paths
SILVER_PATH = f"{LAKEHOUSE_ROOT_PATH}/Tables/silver_crypto"
GOLD_PATH   = f"{LAKEHOUSE_ROOT_PATH}/Tables/gold_daily_summary"

# ==============================================================================
# 2. READ SILVER
# ==============================================================================
print(f"Reading Silver data from: {SILVER_PATH}")
df_silver = spark.read.format("delta").load(SILVER_PATH)

# ==============================================================================
# 3. TRANSFORM TO GOLD (AGGREGATION)
# ==============================================================================
# We group by Symbol and Date to get a daily summary
df_gold = df_silver.groupBy(
    col("symbol"),
    col("name"),
    to_date(col("last_updated_time")).alias("report_date")
).agg(
    avg("current_price").alias("avg_price"),
    max("current_price").alias("max_price"),
    min("current_price").alias("min_price"),
    max("market_cap").alias("max_market_cap")
)

# Add Audit Column
df_gold = df_gold.withColumn("processed_at", current_timestamp())

# ==============================================================================
# 4. WRITE TO GOLD
# ==============================================================================
print(f"Writing Gold data to: {GOLD_PATH}...")

# Write as Delta Table
df_gold.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(GOLD_PATH)

print("✅ SUCCESS: Gold table 'gold_daily_summary' created.")

# ==============================================================================
# 5. VERIFY
# ==============================================================================
print("\n--- GOLD DATA PREVIEW ---")
display(spark.read.format("delta").load(GOLD_PATH).show(5))


StatementMeta(, 5f48ff42-54cb-4f72-ba00-8122e6506283, 3, Finished, Available, Finished)

Reading Silver data from: abfss://DE_Project_Portfolio@onelake.dfs.fabric.microsoft.com/Lh_Medallion.Lakehouse/Files/Tables/silver_crypto
Writing Gold data to: abfss://DE_Project_Portfolio@onelake.dfs.fabric.microsoft.com/Lh_Medallion.Lakehouse/Files/Tables/gold_daily_summary...
✅ SUCCESS: Gold table 'gold_daily_summary' created.

--- GOLD DATA PREVIEW ---
+------+--------+-----------+---------+---------+---------+--------------+--------------------+
|symbol|    name|report_date|avg_price|max_price|min_price|max_market_cap|        processed_at|
+------+--------+-----------+---------+---------+---------+--------------+--------------------+
|  dash|    Dash| 2026-01-19|    84.21|    84.21|    84.21|    1052937335|2026-01-19 06:02:...|
|  paxg|PAX Gold| 2026-01-19|  4679.08|  4679.08|  4679.08|    1793273654|2026-01-19 06:02:...|
|   ton| Toncoin| 2026-01-19|     1.59|     1.59|     1.59|    3851515704|2026-01-19 06:02:...|
|  doge|Dogecoin| 2026-01-19| 0.127074| 0.127074| 0.127074|   213